# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_api_df = pd.read_csv("Output/weather.csv", dtype="object", encoding="utf-8")
weather_api_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed
0,Punta Arenas,CL,-53.15,-70.92,1594685099,90,80,35.6,8.05
1,Nieuw Amsterdam,SR,5.88,-55.08,1594685100,40,83,80.6,8.05
2,Bredasdorp,ZA,-34.53,20.04,1594685101,70,87,46.4,13.87
3,Cape Town,ZA,-33.93,18.42,1594685101,75,76,52.0,8.05
4,Barsovo,RU,61.17,73.17,1594685101,1,87,57.2,8.95


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = weather_api_df[["Latitude", "Longitude"]].astype(float)
humidity = weather_api_df["Humidity"].astype(float)
max_humidity = humidity.max()
locations.head()

,Latitude,Longitude
0,-53.15,-70.92
1,5.88,-55.08
2,-34.53,20.04
3,-33.93,18.42
4,61.17,73.17


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [6]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)
plt.savefig("VacationHeatMap.png")
fig


Figure(layout=FigureLayout(height='420px'))

<Figure size 432x288 with 0 Axes>

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
vacation_spots = weather_api_df.loc[(weather_api_df["Max Temp"] > 70) & (weather_api_df["Max Temp"] < 80) & (weather_api_df["Cloudiness"] ==0), :]
vacation_spots = vacation_spots.dropna(how="any")
vacation_spots = vacation_spots.reset_index()
del vacation_spots ["index"]
vacation_spots.head()

In [8]:
hotel_df = weather_api_df
hotel_df["Hotel Name"] = ""
hotel_df.head()

,City,Country,Latitude,Longitude,Date,Cloudiness,Humidity,Max Temp,Wind Speed,Hotel Name
0,Punta Arenas,CL,-53.15,-70.92,1594685099,90,80,35.6,8.05,
1,Nieuw Amsterdam,SR,5.88,-55.08,1594685100,40,83,80.6,8.05,
2,Bredasdorp,ZA,-34.53,20.04,1594685101,70,87,46.4,13.87,
3,Cape Town,ZA,-33.93,18.42,1594685101,75,76,52.0,8.05,
4,Barsovo,RU,61.17,73.17,1594685101,1,87,57.2,8.95,


In [ ]:
hotel_info = []

for index, row in hotel_df.iterrows():
    latitude = row["Latitude"]
    longitude = row["Longitude"]
    
    params= {
        "location": f"{latitude},{longitude}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   
    hotel_info = requests.get(url, params=params)
    hotel_info = hotel_info.json()
    hotel_info.head()

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Latitude", "Longitude"]]

In [ ]:
# Add marker layer ontop of heat map
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=[f"Hotel Location: {hotel}" for Hotel in hotel_df]
# Display figure
fig = gmaps.figure()
fig.add_layer(hotel_layer)

fig